**[Colab Notebook](https://colab.research.google.com/drive/1lg2Z5DE8qt6EGc3cDO8oiTeSG-jrjtn0#scrollTo=8EAtzLF34Aor) for the same**

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import torch
import pandas as pd
from torch import nn, optim
import cv2
from torchvision import datasets, transforms, models
from torch.utils.data.sampler import SubsetRandomSampler

def load_data(directory):
    transform = transforms.Compose([transforms.Resize(224),
                                       transforms.ToTensor(),
                                       ])
    train_data = datasets.ImageFolder(directory, transform=transform)
    labels = pd.read_csv(directory + 'train.csv')

    trainloader = torch.utils.data.DataLoader((train_data, labels), batch_size=64)

    return trainloader
  
def imgtopix(directory):
    class_ = os.listdir(directory)
    pixels = []
    labels = []
    for label in class_:
        images = os.listdir(directory+label+'/')
        for image in images:
          img = cv2.imread(directory+label+'/'+image)
          img = cv2.resize(img, (224, 224))
          img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
          pixels.append(torch.tensor(img.tolist()))
          labels.append(labels)
    dataframe = pd.DataFrame({'pixels': pixels, 'labels': labels})      
    return dataframe
# Training data directory
LOCATION = '/content/drive/My Drive/ThingsToDo/Intel/data/train-data/'
trainloader = load_data(LOCATION)

# Pretrained Resnet model
model = models.resnet50(pretrained=True)

# Mentioning our device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Freezing layers from getting trained
for param in model.parameters():
    param.requires_grad = False

# Modifying fully connected layers
model.fc = nn.Sequential(nn.Linear(in_features=2048, out_features=512),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(in_features=512, out_features=6),
                                 nn.LogSoftmax(dim=1))
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.003)

# Training the model starts
epochs = 10
steps = 20
running_loss = 0
print_every = 10
train_losses, test_losses = [], []
for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1

        optimizer.zero_grad()
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in validationloader:
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    test_loss += batch_loss.item()

                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            train_losses.append(running_loss / len(trainloader))
            test_losses.append(test_loss / len(testloader))
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            model.train()
torch.save(model, 'model.pth')

In [8]:
import os
os.listdir('/content/drive/My Drive/ThingsToDo/Intel/data/train-data/')

['train.csv', '0', '5', '2', '3', '1', '4']

In [0]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/ThingsToDo/Intel/data/train-data/train.csv')